In [20]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [21]:
print("hello")

hello


In [22]:
training_data = datasets.FashionMNIST(root="data", train=True, transform=ToTensor(), download=True)
test_data = datasets.FashionMNIST(root="data", train=False, transform=ToTensor(), download=True)

In [23]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 18)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=18, bias=True)
  )
)


In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [26]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [30]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [31]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.885360  [    0/60000]
loss: 2.841220  [ 6400/60000]
loss: 2.788033  [12800/60000]
loss: 2.758802  [19200/60000]
loss: 2.707522  [25600/60000]
loss: 2.631382  [32000/60000]
loss: 2.625377  [38400/60000]
loss: 2.533580  [44800/60000]
loss: 2.501811  [51200/60000]
loss: 2.408904  [57600/60000]
Test Error: 
 Accuracy: 29.7%, Avg loss: 2.378486 

Epoch 2
-------------------------------
loss: 2.397541  [    0/60000]
loss: 2.366842  [ 6400/60000]
loss: 2.237725  [12800/60000]
loss: 2.259305  [19200/60000]
loss: 2.174202  [25600/60000]
loss: 2.045454  [32000/60000]
loss: 2.119141  [38400/60000]
loss: 1.985229  [44800/60000]
loss: 1.990976  [51200/60000]
loss: 1.884779  [57600/60000]
Test Error: 
 Accuracy: 53.1%, Avg loss: 1.879444 

Epoch 3
-------------------------------
loss: 1.928611  [    0/60000]
loss: 1.881359  [ 6400/60000]
loss: 1.733259  [12800/60000]
loss: 1.777499  [19200/60000]
loss: 1.649884  [25600/60000]
loss: 1.601091  [32000/600